##### Copyright 2020 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Нормировки

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/addons/tutorials/layers_normalizations"><img src="https://www.tensorflow.org/images/tf_logo_32px.png"> Посмотреть на TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/addons/blob/master/docs/tutorials/layers_normalizations.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png"> Запустить в Google Colab</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/addons/blob/master/docs/tutorials/layers_normalizations.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"> Посмотреть источник на GitHub</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/addons/docs/tutorials/layers_normalizations.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png"> Скачать блокнот</a></td>
</table>


## обзор

Этот блокнот дает краткое введение в [уровни нормализации](https://github.com/tensorflow/addons/blob/master/tensorflow_addons/layers/normalizations.py) TensorFlow. В настоящее время поддерживаются следующие слои:

- **Групповая нормализация** (аддоны TensorFlow)
- **Нормализация экземпляров** (аддоны TensorFlow)
- **Нормализация слоя** (TensorFlow Core)

Основная идея этих слоев - нормализовать вывод слоя активации, чтобы улучшить конвергенцию во время обучения. В отличие от [пакетной нормализации](https://keras.io/layers/normalization/) эти нормализации не работают с пакетами, вместо этого они нормализуют активации одного образца, что делает их пригодными и для повторяющихся новых сетей.

Обычно нормализация выполняется путем вычисления среднего значения и стандартного отклонения подгруппы во входном тензоре. К этому также можно применить масштаб и коэффициент смещения.

$ y_ {i} = \ frac {\ gamma (x_ {i} - \ mu)} {\ sigma} + \ beta $

$ y $: вывод

$ x $: вход

$ \ gamma $: коэффициент масштабирования

$ \ mu $: среднее

$ \ sigma $: стандартное отклонение

$ \ beta $: коэффициент смещения

Следующее изображение демонстрирует разницу между этими методами. Каждый субплот показывает входной тензор с N в качестве оси пакета, C в качестве оси канала и (H, W) в качестве пространственных осей (например, высота и ширина изображения). Пиксели синего цвета нормализуются с помощью того же среднего значения и дисперсии, которые вычисляются путем суммирования значений этих пикселей.

![](https://github.com/shaohua0116/Group-Normalization-Tensorflow/raw/master/figure/gn.png)

Источник: (https://arxiv.org/pdf/1803.08494.pdf)

Веса гамма и бета обучаемы на всех уровнях нормализации, чтобы компенсировать возможную потерю репрезентативной способности. Вы можете активировать эти факторы, установив `center` или флаг `scale` на `True` . Конечно, вы можете использовать `initializers` , `constraints` и `regularizer` для `beta` и `gamma` чтобы настроить эти значения в процессе обучения. 

## Настроить

### Установите Tensorflow 2.0 и Tensorflow-Addons

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa

### Подготовка набора данных

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## Учебник по нормализации группы

### Введение

Групповая нормализация (GN) разделяет каналы ваших входов на более мелкие подгруппы и нормализует эти значения на основе их среднего значения и дисперсии. Поскольку GN работает на одном примере, этот метод не зависит от размера пакета.

GN экспериментально забил закрытый для пакетной нормализации в задачах классификации изображений. Может быть выгодно использовать GN вместо нормализации партии, если ваш общий размер пакета batch_size низкий, что может привести к плохой производительности нормализации партии

### Пример Разделение 10 каналов после слоя Conv2D на 5 подгрупп в стандартной настройке «последний канал»:

In [ ]:
model = tf.keras.models.Sequential([
  # Reshape into "channels last" setup.
  tf.keras.layers.Reshape((28,28,1), input_shape=(28,28)),
  tf.keras.layers.Conv2D(filters=10, kernel_size=(3,3),data_format="channels_last"),
  # Groupnorm Layer
  tfa.layers.GroupNormalization(groups=5, axis=3),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_test, y_test)

## Учебник по нормализации экземпляров

### Введение

Нормализация экземпляра - это особый случай нормализации группы, когда размер группы равен размеру канала (или размеру оси).

Экспериментальные результаты показывают, что нормализация экземпляра хорошо работает при передаче стиля при замене пакетной нормализации. Недавно нормализация экземпляров также использовалась в качестве замены для нормализации партии в GAN.

### пример

Применение InstanceNormalization после слоя Conv2D и использование унифицированного масштабированного инициализированного масштаба и коэффициента смещения.

In [ ]:
model = tf.keras.models.Sequential([
  # Reshape into "channels last" setup.
  tf.keras.layers.Reshape((28,28,1), input_shape=(28,28)),
  tf.keras.layers.Conv2D(filters=10, kernel_size=(3,3),data_format="channels_last"),
  # LayerNorm Layer
  tfa.layers.InstanceNormalization(axis=3, 
                                   center=True, 
                                   scale=True,
                                   beta_initializer="random_uniform",
                                   gamma_initializer="random_uniform"),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_test, y_test)

## Учебник по нормализации слоя

### Введение

Нормализация уровня - это особый случай нормализации группы, когда размер группы равен 1. Среднее значение и стандартное отклонение рассчитывается по всем активациям одной выборки.

Экспериментальные результаты показывают, что нормализация слоев хорошо подходит для рекуррентных нейронных сетей, поскольку она работает в пакетном режиме независимо.

### пример

Применение нормализации слоя после слоя Conv2D и использование масштаба и коэффициента смещения. 

In [ ]:
model = tf.keras.models.Sequential([
  # Reshape into "channels last" setup.
  tf.keras.layers.Reshape((28,28,1), input_shape=(28,28)),
  tf.keras.layers.Conv2D(filters=10, kernel_size=(3,3),data_format="channels_last"),
  # LayerNorm Layer
  tf.keras.layers.LayerNormalization(axis=1 , center=True , scale=True),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_test, y_test)

## Литература

[Норма слоя](https://arxiv.org/pdf/1607.06450.pdf)

[Норма экземпляра](https://arxiv.org/pdf/1607.08022.pdf)

[Групповая норма](https://arxiv.org/pdf/1803.08494.pdf)

[Полный обзор нормализации](http://mlexplained.com/2018/11/30/an-overview-of-normalization-methods-in-deep-learning/)